In [2]:
import sys
import os
import inspect
# currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = "d:\\code\\Python\\myfile\\smart_computer"

sys.path.insert(0, parentdir) 

import torch
import numpy as np
from pyhessian.hessian import hessian
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
import torch.nn.functional as F
import torch.nn as nn

class SimpleCNN(nn.Module):
    def __init__(self, input_dim = 400, hidden_dims = [120,84], output_dim=10):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)

        # for now, we hard coded this network
        # i.e. we fix the number of hidden layers i.e. 2 layers
        self.fc1 = nn.Linear(input_dim, hidden_dims[0])
        self.fc2 = nn.Linear(hidden_dims[0], hidden_dims[1])
        self.classifier = nn.Linear(hidden_dims[1], output_dim)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # x = F.relu(self.conv1(x))
        # x = F.relu(self.conv2(x))      
        x = x.view(-1, 16 * 5 * 5)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        y = self.classifier(x)
        return y
device = 'cuda:0'       
model = SimpleCNN().to(device)

model.load_state_dict(torch.load("logs/max.pt"))
transform_all = transforms.Compose([
transforms.ToTensor(),
transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
train_dataset = datasets.CIFAR10('../data/cifar10/', train=True, transform=transform_all)
test_dataset = datasets.CIFAR10('../data/cifar10/', train=False, transform=transform_all)

train_loader = DataLoader(train_dataset, batch_size=512,shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=512)


d:\Program Files\Anaconda3\envs\FLtorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model.eval()
criterion = torch.nn.CrossEntropyLoss()
for inputs, targets in train_loader:
    break
inputs, targets = inputs.cuda(), targets.cuda()
hessian_comp = hessian(model, criterion, data=(inputs, targets), device=device)
trace = hessian_comp.trace_layer()

10 10
conv1 torch.Size([6, 3, 5, 5])
pool torch.Size([6])
conv2 torch.Size([16, 6, 5, 5])
fc1 torch.Size([16])
fc2 torch.Size([120, 400])
classifier torch.Size([120])


d:\Program Files\Anaconda3\envs\FLtorch\lib\site-packages\torch\autograd\__init__.py:156: UserWarning: Using backward() with create_graph=True will create a reference cycle between the parameter and its gradient which can cause a memory leak. We recommend using autograd.grad when creating the graph to avoid this. If you have to use this function, make sure to reset the .grad fields of your parameters to None after use to break the cycle and avoid the leak. (Triggered internally at  ..\torch\csrc\autograd\engine.cpp:976.)
  allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag


In [7]:
print(model.named_parameters().shape)
for name, param in model.named_parameters():
  
    print(name, param.size())

AttributeError: 'generator' object has no attribute 'shape'

In [4]:
trace

[{'layer_name': 'conv1', 'trace': 42.4210205078125},
 {'layer_name': 'conv2', 'trace': 79.0521011352539},
 {'layer_name': 'fc2', 'trace': 127.43635559082031}]

In [3]:
import torch

# 创建一个简单的线性回归模型
class LinearRegression(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.weight = torch.nn.Parameter(torch.randn(2))
        self.bias = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):
        return torch.sum(self.weight * x, dim=1) + self.bias

# 准备训练数据
x = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
y = torch.tensor([3.0, 7.0])

# 创建模型和优化器
model = LinearRegression()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# 计算梯度并更新参数
loss_fn = torch.nn.MSELoss()

# 生成随机整数张量列表
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
v = [
    torch.randint_like(p, high=2, device=device)
    for p in model.parameters()
]

# 计算梯度
y_pred = model(x)
loss = loss_fn(y_pred, y)
gradsH = torch.autograd.grad(loss, y_pred, only_inputs=True)[0]
params = list(model.parameters())
hv = torch.autograd.grad(gradsH, params, grad_outputs=v, only_inputs=True, retain_graph=True)

hv


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn